# Data Preparation and Engineering
Author: Oscar Janossy   
Date: 2025-05-19  
Description: Preparation of dataset for machine learning models

The preprocessing part is done using ressources from the following repository:  
**Modern Time Series Forecasting with Python**  
- **Repository:** [PacktPublishing/Modern-Time-Series-Forecasting-with-Python](https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python)  
- **Author:** Manu Joseph & Jeffrey Tackes
- **License:** MIT License  
- **Accessed on:** 2025-05-17 


## Import librairies and dataset

In [18]:
# Core Python & System
import os
import re
import sys

# Type & Utility Annotations
from typing import List, Tuple 

# Date & Calendar Utilities
import holidays
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

# Data & Utilities
import numpy as np
import pandas as pd
from pandas.api.types import is_list_like


# Feature Engineering
from feature_engine.creation import CyclicalFeatures

In [19]:
### Import Dataset

# Path Setup
directory = os.getcwd()
sys.path.insert(0, directory)
directory = '/Users/oscar/Documents/VS Code/Thesis/'

data = pd.read_csv(directory + '/data/data_ml_new.csv', parse_dates=['start_date'], index_col=0)
data.head()

,Imbalance,ISP (lag1),Day-ahead Spot Price,WIND_ONSHORE - D-1,WIND_ONSHORE - ID,WIND_ONSHORE - CURRENT,SOLAR - D-1,SOLAR - ID,SOLAR - CURRENT,WIND Total D-1,...,SOLAR DEVIATION CURRENT (lag1),WIND DEVIATION (lag1),WIND DEVIATION CURRENT (lag1),LOAD DEVIATION (lag1),CONSUMPTION DEVIATION (lag1),PROGRAMMABLE RATIO D-1,NUCLEAR - SHARE % (lag1),GAS -SHARE % (lag1),SOLAR - SHARE % (lag1),WIND - SHARE % (lag1)
start_date,,,,,,,,,,,,,,,,,,,,,
2017-12-31 23:00:00+00:00,-377.32,NaN,6.74,10081.0,9839.0,9839.0,0.0,0.0,0.0,10081.0,...,NaN,NaN,NaN,NaN,NaN,0.136,NaN,NaN,NaN,NaN
2017-12-31 23:30:00+00:00,-468.52,29.22,6.74,10047.5,9820.5,9820.5,0.0,0.0,0.0,10047.5,...,0.0,9.0,9.0,1388.0,-18.0,0.139,0.682,0.039,0.0,0.169
2018-01-01 00:00:00+00:00,-1467.79,18.11,4.74,10014.0,9802.0,9802.0,0.0,0.0,0.0,10014.0,...,0.0,-25.0,-25.0,1562.0,-23.0,0.149,0.639,0.041,0.0,0.172
2018-01-01 00:30:00+00:00,-1467.09,20.49,4.74,9805.0,9717.5,9717.5,0.0,0.0,0.0,9805.0,...,0.0,-67.5,-67.5,1736.0,157.0,0.149,0.630,0.042,0.0,0.175
2018-01-01 01:00:00+00:00,-1230.20,23.22,3.66,9596.0,9633.0,9633.0,0.0,0.0,0.0,9596.0,...,0.0,-58.5,-58.5,1315.0,-688.0,0.152,0.634,0.042,0.0,0.176


In [20]:
# Set target variable
target = 'Imbalance'

## Feature enginneering part
Create lag features and temporal features


In [21]:
#Creat a copy of the data
full_df = data.copy()
full_df.reset_index(drop=False, inplace=True)

## Data preprocessing

Create the functions

In [22]:

def add_lags(
    df: pd.DataFrame,
    lags: List[int],
    column: str,
) -> Tuple[pd.DataFrame, List]:
    """Create Lags for the column provided and adds them as other columns in the provided dataframe

    Args:
        df (pd.DataFrame): The dataframe in which features needed to be created
        lags (List[int]): List of lags to be created
        column (str): Name of the column to be lagged
        ts_id (str, optional): Column name of Unique ID of a time series to be grouped by before applying the lags.
            If None assumes dataframe only has a single timeseries. Defaults to None.
        use_32_bit (bool, optional): Flag to use float32 or int32 to reduce memory. Defaults to False.

    Returns:
        Tuple(pd.DataFrame, List): Returns a tuple of the new dataframe and a list of features which were added
    """
    assert is_list_like(lags), "`lags` should be a list of all required lags"
    assert (
        column in df.columns
    ), "`column` should be a valid column in the provided dataframe"
    #_32_bit_dtype = _get_32_bit_dtype(df[column])
    
        # Assuming just one unique time series in dataset
        # if use_32_bit and _32_bit_dtype is not None:
    col_dict = {
        f"{column}_lag_{l}": df[column].shift(l).astype('float32')
        for l in lags
    }
        
        
    df = df.assign(**col_dict)
    added_features = list(col_dict.keys())
    return df, added_features

# adapted from gluonts
def time_features_from_frequency_str(freq_str: str) -> List[str]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.

    Parameters
    ----------

    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.

    """

    features_by_offsets = {
        offsets.YearBegin: [],
        offsets.YearEnd: [],
        offsets.MonthBegin: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
        ],
        offsets.MonthEnd: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
        ],
        offsets.Week: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
            "Is_month_start",
            "Week",
        ],
        offsets.Day: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
            "Is_month_start",
            "Week",
            "Day",
            "Dayofweek",
            "Dayofyear",
        ],
        offsets.BusinessDay: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
            "Is_month_start",
            "Week",
            "Day",
            "Dayofweek",
            "Dayofyear",
        ],
        offsets.Hour: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
            "Is_month_start",
            "Week",
            "Day",
            "Dayofweek",
            "Dayofyear",
            "Hour",
        ],
        offsets.Minute: [
            "Month",
            "Quarter",
            "Is_quarter_end",
            "Is_quarter_start",
            "Is_year_end",
            "Is_year_start",
            "Is_month_start",
            "Week",
            "Day",
            "Dayofweek",
            "Dayofyear",
            "Hour",
            "Minute",
        ],
    }

    offset = to_offset(freq_str)

    for offset_type, feature in features_by_offsets.items():
        if isinstance(offset, offset_type):
            return feature

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}

    The following frequencies are supported:

        Y, YS   - yearly
            alias: A
        M, MS   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
    """
    raise RuntimeError(supported_freq_msg)

# adapted from fastai
def add_temporal_features(
    df: pd.DataFrame,
    field_name: str,
    frequency: str,
    add_elapsed: bool = True,
    prefix: str = None,
    drop: bool = True,
    use_32_bit: bool = False,
) -> Tuple[pd.DataFrame, List]:
    """Adds columns relevant to a date in the column `field_name` of `df`.

    Args:
        df (pd.DataFrame): Dataframe to which the features need to be added
        field_name (str): The date column which should be encoded using temporal features
        frequency (str): The frequency of the date column so that only relevant features are added.
            If frequency is "Weekly", then temporal features like hour, minutes, etc. doesn't make sense.
        add_elapsed (bool, optional): Add time elapsed as a monotonically increasing function. Defaults to True.
        prefix (str, optional): Prefix to the newly created columns. If left None, will use the field name. Defaults to None.
        drop (bool, optional): Flag to drop the data column after feature creation. Defaults to True.
        use_32_bit (bool, optional): Flag to use float32 or int32 to reduce memory. Defaults to False.

    Returns:
        Tuple[pd.DataFrame, List]: Returns a tuple of the new dataframe and a list of features which were added
    """
    field = df[field_name]
    prefix = (re.sub("[Dd]ate$", "", field_name) if prefix is None else prefix) 
    attr = time_features_from_frequency_str(frequency)
    _32_bit_dtype = "int32"
    added_features = []
    for n in attr:
        if n == "Week":
            continue
        df[prefix + n] = (
            getattr(field.dt, n.lower()).astype(_32_bit_dtype)
            if use_32_bit
            else getattr(field.dt, n.lower())
        )
        added_features.append(prefix + n)
    # Pandas removed `dt.week` in v1.1.10
    if "Week" in attr:
        week = (
            field.dt.isocalendar().week
            if hasattr(field.dt, "isocalendar")
            else field.dt.week
        )
        df.insert(
            3, prefix + "Week", week.astype(_32_bit_dtype) if use_32_bit else week
        )
        added_features.append(prefix + "Week")
    if add_elapsed:
        mask = ~field.isna()
        df[prefix + "Elapsed"] = np.where(
            mask, field.values.astype(np.int64) // 10**9, None
        )
        if use_32_bit:
            if df[prefix + "Elapsed"].isnull().sum() == 0:
                df[prefix + "Elapsed"] = df[prefix + "Elapsed"].astype("int32")
            else:
                df[prefix + "Elapsed"] = df[prefix + "Elapsed"].astype("float32")
        added_features.append(prefix + "Elapsed")
    if drop:
        df.drop(field_name, axis=1, inplace=True)
    return df, added_features


Add lagged values

In [24]:
#Too big for github
lags = [i for i in range(1, 100)]

#Use only lags used in the paper    
lags = [1,2,3,4,5,6, 48, 96,]

full_df, lagged_imbalance = add_lags(full_df, lags=lags, column='Imbalance')

Add temporal features

In [25]:
full_df, added_features = add_temporal_features(
        full_df,
        field_name="start_date",
        frequency="30min",
        add_elapsed=True,
        prefix="",
        drop=False,
        use_32_bit=True,
    )
print(f"Features Created: {','.join(added_features)}")

Features Created: Month,Quarter,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Is_month_start,Day,Dayofweek,Dayofyear,Hour,Minute,Week,Elapsed


Add the 1/2 hour of the day feature

In [26]:
full_df["1/2 Hour of the day"] = full_df["start_date"].dt.hour * 2 + full_df["start_date"].dt.minute // 30 + 1

Add weekday, weekend and holidays

In [27]:
# Add the weekend/weekday feature
full_df["day_type"] = np.where(full_df["start_date"].dt.dayofweek.isin([5, 6]), "weekend", "weekday")
# Add the holiday feature
fr_holidays = holidays.FR()  # this is a dict-like object


full_df['day_type'] = full_df.apply(
    lambda row: 'holiday' if row['start_date'].date() in fr_holidays else row['day_type'],
    axis=1
)
full_df['day_type'] = pd.Categorical(full_df['day_type'], categories=['weekday', 'weekend', 'holiday'])



In [28]:
full_df = pd.concat([full_df, pd.get_dummies(full_df['day_type'], prefix='day', dtype=int)], axis=1)
full_df.drop(columns=['day_type'], inplace=True)

### Cyclical encoding of the time features

In [29]:
# Cyclical encoding
# ==============================================================================
features_to_encode = [
    "Month",
    "Week",
    "Dayofweek",
    "1/2 Hour of the day",
]
max_values = {
    "Month": 12,
    "Week": 52,
    "Dayofweek": 7,
    "1/2 Hour of the day": 48,
}
cyclical_encoder = CyclicalFeatures(
    variables     = features_to_encode,
    max_values    = max_values,
    drop_original = True
)


full_df = cyclical_encoder.fit_transform(full_df)

#Put back 1/2 hour of the day
full_df["1/2 Hour of the day"] = full_df["start_date"].dt.hour * 2 + full_df["start_date"].dt.minute // 30 + 1

full_df.head(3)

,start_date,Imbalance,ISP (lag1),Day-ahead Spot Price,WIND_ONSHORE - D-1,WIND_ONSHORE - ID,WIND_ONSHORE - CURRENT,SOLAR - D-1,SOLAR - ID,SOLAR - CURRENT,...,day_holiday,Month_sin,Month_cos,Week_sin,Week_cos,Dayofweek_sin,Dayofweek_cos,1/2 Hour of the day_sin,1/2 Hour of the day_cos,1/2 Hour of the day
0,2017-12-31 23:00:00+00:00,-377.32,NaN,6.74,10081.0,9839.0,9839.0,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-1.305262e-01,0.991445,47
1,2017-12-31 23:30:00+00:00,-468.52,29.22,6.74,10047.5,9820.5,9820.5,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-2.449294e-16,1.000000,48
2,2018-01-01 00:00:00+00:00,-1467.79,18.11,4.74,10014.0,9802.0,9802.0,0.0,0.0,0.0,...,1,5.000000e-01,0.866025,1.205367e-01,0.992709,0.000000,1.00000,1.305262e-01,0.991445,1


In [30]:
# Fill the missing values of the lagged features using backward fill
full_df.bfill(inplace=True)

In [31]:
full_df.describe()

,Imbalance,ISP (lag1),Day-ahead Spot Price,WIND_ONSHORE - D-1,WIND_ONSHORE - ID,WIND_ONSHORE - CURRENT,SOLAR - D-1,SOLAR - ID,SOLAR - CURRENT,WIND Total D-1,...,day_holiday,Month_sin,Month_cos,Week_sin,Week_cos,Dayofweek_sin,Dayofweek_cos,1/2 Hour of the day_sin,1/2 Hour of the day_cos,1/2 Hour of the day
count,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,122640.000000,...,122640.000000,1.226400e+05,1.226400e+05,1.226400e+05,1.226400e+05,1.226400e+05,1.226400e+05,1.226400e+05,1.226400e+05,122640.000000
mean,36.976237,93.778548,94.490298,4336.764088,4353.035022,4366.609119,1673.052542,1674.392622,1687.358448,4276.635255,...,0.030137,-4.703194e-03,-2.764062e-03,3.302375e-04,2.719750e-03,-2.444953e-17,-1.367320e-17,2.896864e-20,-1.293160e-16,24.500000
std,474.664956,125.044346,104.358241,3253.723683,3335.841621,3345.268178,2680.337252,2671.076665,2681.795981,3361.665328,...,0.170965,7.060554e-01,7.081413e-01,7.061685e-01,7.080443e-01,7.071097e-01,7.071097e-01,7.071097e-01,7.071097e-01,13.853456
min,-2895.150000,-3940.110000,-134.940000,185.090000,131.760000,149.750000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-9.749279e-01,-9.009689e-01,-1.000000e+00,-1.000000e+00,1.000000
25%,-258.172500,31.980000,36.370000,1995.597500,1946.030000,1942.627500,0.000000,0.000000,0.000000,1904.070000,...,0.000000,-8.660254e-01,-8.660254e-01,-6.631227e-01,-6.631227e-01,-7.818315e-01,-9.009689e-01,-7.071068e-01,-7.071068e-01,12.750000
50%,34.675000,54.550000,57.845000,3313.520000,3262.030000,3277.620000,0.000000,0.010000,0.060000,3231.290000,...,0.000000,-2.449294e-16,-1.836970e-16,1.224647e-16,6.123234e-17,0.000000e+00,-2.225209e-01,-6.123234e-17,-6.123234e-17,24.500000
75%,330.862500,113.842500,107.170000,5720.520000,5787.547500,5829.680000,2776.442500,2793.267500,2811.717500,5649.420000,...,0.000000,5.000000e-01,8.660254e-01,6.631227e-01,7.485107e-01,7.818315e-01,6.234898e-01,7.071068e-01,7.071068e-01,36.250000
max,3558.840000,3866.420000,2987.780000,20156.400000,20269.190000,20256.850000,15323.460000,15313.240000,15499.530000,21003.020000,...,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.749279e-01,1.000000e+00,1.000000e+00,1.000000e+00,48.000000


In [32]:
# Convert all numeric columns to 32-bit data types
numeric_cols = full_df.select_dtypes(include=['float64', 'int64']).columns
full_df[numeric_cols] = full_df[numeric_cols].astype('float32')

In [33]:
full_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122640 entries, 0 to 122639
Data columns (total 63 columns):
 #   Column                                    Non-Null Count   Dtype              
---  ------                                    --------------   -----              
 0   start_date                                122640 non-null  datetime64[ns, UTC]
 1   Imbalance                                 122640 non-null  float32            
 2   ISP (lag1)                                122640 non-null  float32            
 3   Day-ahead Spot Price                      122640 non-null  float32            
 4   WIND_ONSHORE - D-1                        122640 non-null  float32            
 5   WIND_ONSHORE - ID                         122640 non-null  float32            
 6   WIND_ONSHORE - CURRENT                    122640 non-null  float32            
 7   SOLAR - D-1                               122640 non-null  float32            
 8   SOLAR - ID                                12

In [37]:
#Save as csv
full_df.to_csv(directory + '/data/data.csv', index=True, header=True)

In [93]:
full_df

,start_date,Imbalance,ISP (lag1),Day-ahead Spot Price,WIND_ONSHORE - D-1,WIND_ONSHORE - ID,WIND_ONSHORE - CURRENT,SOLAR - D-1,SOLAR - ID,SOLAR - CURRENT,...,day_holiday,Month_sin,Month_cos,Week_sin,Week_cos,Dayofweek_sin,Dayofweek_cos,1/2 Hour of the day_sin,1/2 Hour of the day_cos,1/2 Hour of the day
0,2017-12-31 23:00:00+00:00,-377.32,29.22,6.74,10081.00,9839.00,9839.00,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-1.305262e-01,0.991445,47
1,2017-12-31 23:30:00+00:00,-468.52,29.22,6.74,10047.50,9820.50,9820.50,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-2.449294e-16,1.000000,48
2,2018-01-01 00:00:00+00:00,-1467.79,18.11,4.74,10014.00,9802.00,9802.00,0.0,0.0,0.0,...,1,5.000000e-01,0.866025,1.205367e-01,0.992709,0.000000,1.00000,1.305262e-01,0.991445,1
3,2018-01-01 00:30:00+00:00,-1467.09,20.49,4.74,9805.00,9717.50,9717.50,0.0,0.0,0.0,...,1,5.000000e-01,0.866025,1.205367e-01,0.992709,0.000000,1.00000,2.588190e-01,0.965926,2
4,2018-01-01 01:00:00+00:00,-1230.20,23.22,3.66,9596.00,9633.00,9633.00,0.0,0.0,0.0,...,1,5.000000e-01,0.866025,1.205367e-01,0.992709,0.000000,1.00000,3.826834e-01,0.923880,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122635,2024-12-29 20:30:00+00:00,-83.22,153.58,118.59,2198.20,1378.61,1385.96,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-7.071068e-01,0.707107,42
122636,2024-12-29 21:00:00+00:00,39.56,169.49,119.10,2320.47,1424.86,1432.62,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-6.087614e-01,0.793353,43
122637,2024-12-29 21:30:00+00:00,-320.48,-17.38,119.10,2365.96,1450.69,1459.84,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-5.000000e-01,0.866025,44
122638,2024-12-29 22:00:00+00:00,59.24,164.58,103.18,2411.45,1476.52,1487.07,0.0,0.0,0.0,...,0,-2.449294e-16,1.000000,-2.449294e-16,1.000000,-0.781831,0.62349,-3.826834e-01,0.923880,45
